In [ ]:
import numpy as np
import cv2
import time
from picamera2 import Picamera2
from pinkylib import Pinky, Camera, Yolo

In [ ]:
APPLE = 0
target_id_list = [8, 7, 11, 9]
target_id_index = 0
apple_ids = [7]
turn_right_ids = [8, 11]
end_id = 9
pose = {
    8 : [-0.55, 10],
    7 : [2.6, 14],
    11 : [-0.98, 16],
    9 : [1.9, 9]
}

In [ ]:
pinky = Pinky()
pinky.start_motor()
pinky.enable_motor()

In [ ]:
cam = Camera()
cam.start()
cam.set_calibration()

In [ ]:
yolo = Yolo()
yolo.set_model("./best.pt")

In [ ]:
def find_aruco():
    pinky.turn_right(30, 30)
    time.sleep(0.2)
    pinky.stop()
    time.sleep(1)

In [ ]:
def turn_right():
    pinky.turn_right(30, 30)
    time.sleep(0.4)
    pinky.stop()

In [ ]:
def turn_180():
    pinky.turn_right(30, 30)
    time.sleep(1)
    pinky.stop()

In [ ]:
def find_apple(frame):
    global APPLE
    pinky.stop()
    clss, frame = yolo.detect_yolo(frame)
    for i in clss:
        if i == 0:
            APPLE += 1

    return frame

In [ ]:
pinky.buzzer_start()
pinky.buzzer(2)

while True:
    L, R = 0, 0
    
    frame = cam.get_frame()
    
    target_id = target_id_list[target_id_index]
    target_x, target_z = pose[target_id]
    print(f"target id is {target_id}")

    aruco_pose = cam.detect_aruco_target(frame, target_id)
    print(aruco_pose)
    
    if aruco_pose is not None:
        aruco_x, aruco_z = aruco_pose[0], aruco_pose[2]
        if target_z > aruco_z and target_x < aruco_x + 0.3 and target_x > aruco_x - 0.3:
            print("stop")
            if target_id in turn_right_ids:
                print("trun right")
                turn_right()
            elif target_id  in apple_ids:
                print("detect apple")
                frame = find_apple(frame)
                print(f"현재 인식한 사과 {APPLE} 개")
                print("turn 180")
                turn_180()
            elif target_id == end_id:
                print("fin")
                pinky.stop()
                break
            target_id_index += 1
        else:
            if target_z < aruco_z:
                L, R = 25, 25
            if target_x > aruco_x + 0.3:
                L = 30
            elif target_x < aruco_x - 0.3:
                R = 30
    else:
        print("find aruco")
        find_aruco()
    
    pinky.move(L, R)

    cam.display_jupyter(frame)
    time.sleep(0.01)


pinky.buzzer(2)